# Genetic Algorithms

The idea of the genetic algorithm is to select the best solutions to a problem and make them perpetuate. We can define the following as a pseudocode of the AG:
- ** ENTRANCE **: Initial population (random)
                 Fitness Function
                 Stopping Criteria
- ** REPEAT ** (until the stop criterion is met):
- ** STEP 1: ** Apply the fitness function to each individual
- ** STEP 2: ** Select the best x individuals
- ** STEP 3: ** Reproduction
         - Apply the crossover to a pair (with prob = p)
         - Apply mutation (with prob = p ’)
- ** STEP 4: ** Form a new population with the children generated
- ** OUTPUT: ** Best individual present in the final generation

## Proposed Problem

Generate a genetic algorithm that learns the Google Chrome DinoRun game. The agent has three possible actions: Jump, Crouch and Do Nothing. Soon, the population of our GA will present multidimensional individuals. Each action has a function that indicates how good it is for a given game state. The current state of the game is given by 10 components (s0, s1, s2, s3, s4, s5, s6, s7, s8, s9).

Therefore, each action of the individual presents a pesp for each component of the state of the game, so that we can evaluate how good the action is given a state. Thus, we have individuals defined as 3x10 matrices.

## Import from Libraries

In [1]:
import numpy as np
import random
from chrome_trex import DinoGame

## Definition of Global Parameters

We define the probability of mutation and crossover of the weights, as well as the number of individuals contained in the population, and the number of which are maintained from one generation to the next.

In [2]:
CHANCE_MUT = .20 # Chance de mutação de uma peso qualquer
CHANCE_CO = .25 # Chance de crossing over de um peso qualquer
NUM_INDIVIDUOS = 15 # Tamnanho da população
NUM_MELHORES = 3 # Número de indivíduos que são mantidos de uma geração para a próxima


## Geração da População Inicial

A população será formada por **n** individuos, e seja primeiramente gerarada de maneira aleatória. Se criará uma lista com **n** individuos de tamanho 3x10 com valores de pesos variando entre -10 e 10.

In [3]:
def populacao_aleatoria(n):
    populacao = []
    
    for i in range(n):
        populacao.append(np.random.uniform(-10,10,(3,10)))
        
    return populacao

## Função de Decisão

Como vimos, dentre as ações disponíveis, temos: Pular, Agachar e Não Fazer Nada. Baseado no estado atual do jogo, o agente tomará a decisão. Para isso define-se duas funções:
- Valor das ações: Se calculará o valor de cada ação para cada individuo. Saida: Para cada indivíduo, um vetor de 3 posições com o valor de cada ação;
- Escolha da Melhor Ação: A melhor ação de cada indivíduo será selecionada, ou seja, a ação com maior valor.



### 1. Valor das Ações

A multiplicação matricial da matriz do indivíduo pelo vetor do estado nos retornará os valores de cada ação.

Por examplo, se o estado tivesse 4 componentes (s0,s1,s2,s3),e as letras de (a *a* j) forem os pesos do indivíduo, e **v0**, **v1** e **v2** os valores das ação matricialmente teriamos: 

$$\begin{bmatrix} a & b & c \\ d & f & g \\ h & i & j \end{bmatrix} \ \begin{bmatrix} s0 \\ s1 \\ s2 \\ s3 \end{bmatrix} = \begin{bmatrix} v0 \\ v1 \\ v2 \end{bmatrix}  $$  

In [4]:
def valor_das_acoes(individuo,estado):
    return individuo @ estado

### 2. Índice da Melhor Ação

Dado o vetor de cada indivíduo com os valores para cada ação, retorna-se o índice do maior valor.

In [5]:
def melhor_jogada(individuo,estado):
    valores = valor_das_acoes(individuo,estado)
    return np.argmax(valores)

### Reprodução e Mutação

Para a reprodução dos indíviduos, a função de crossing over escolherá dois indivíduos da população existente e a partir deles criará um indivíduo novo. Depois disso a função de mutação irá possivelmente alterar esse indivíduo.

### 1. Mutação

Na função de mutação nós vamos possivelmente mutar os pesos do nosso indivíduo com uma probabilidade definida pela variável global CHANCE_MUT. Para fazer com que um evento ocorra com probabilidade p basta gerar um número aleatório entre 0 e 1, e verificar se ele está dentro de um intervalo do tamanho da probabilidade que queremos.

In [6]:
def mutacao(individuo):
    for i in range(3):
        for j in range(10):
            if np.random.uniform(0,1) < CHANCE_MUT:
                individuo[i][j] *= np.random.uniform(-1.5,1.5)

### Reprodução

Vamos definir a nossa “cria” como a cópia de um dos dois indivíduos selecionados e depois trocar cada um dos pesos pelo do outro indivíduo com uma chance dada.

In [7]:
def crossover(individuo1,individuo2):
    filho = individuo1.copy()
    for i in range(3):
        for j in range(10):
            if np.random.uniform(0,1) < CHANCE_CO:
                filho[i][j] = individuo2[i][j]
    return filho

## Fitness Function

A função fitness que avaliará cada individuo ser baseada no score do jogo. Em termos de código, o que a função deve executar é: fazer com que o indivíduo jogue o jogo e depois pegar o score dele.

In [9]:
def calcular_fitness(jogo,individuo):
    jogo.reset()
    while not jogo.game_over:
        estado = jogo.get_state()
        acao = melhor_jogada(individuo,estado)
        jogo.step(acao)
    return jogo.get_score()

## Próxima Geração

Definida a função de fitness, pode-se eliminar os piores indivíduos e reproduzir os melhores. Ou seja, vamos unir o fitness com a reprodução e também selecionar os melhores. Para isso, vamos definir uma função adicional que ordenará a população com base no fitness de cada individuo.

In [10]:
def ordenar_lista(lista, ordenacao, decrescente=True):
    return [x for _, x in sorted(zip(ordenacao,lista), key=lambda p: p[0], reverse = decrescente)]

In [11]:
def proxima_geracao(populacao, fitness):
    ordenados = ordenar_lista(populacao, fitness)
    proxima_ger = ordenados[:NUM_MELHORES]
    
    while len(proxima_ger):
        ind1, ind2 = random.choices(populacao,k=2)
        filho = crossover(ind1,ind2)
        mutacao(filho)
        proxima_ger.append(filho)
    
    return proxima_ger

## Função Main

In [ ]:
num_geracoes = 100
jogo = DinoGame(fps=60)

populacao = populacao_aleatoria(NUM_INDIVIDUOS)

print('ger | fitness\n---+-' + '-'*5*NUM_INDIVIDUOS)

for ger in range(num_geracoes):
    fitness = []
    for ind in populacao:
        fitness.append(calcular_fitness(jogo,ind))
        
    populacao = proxima_geracao(populacao,fitness)
    
    print('{:3} |'.format(ger), ' '.join('{:4d}'.format(s) for s in sorted(fitness, reverse=True)))
    

fitness = []
for ind in populacao:
    fitness.append(calcular_fitness(jogo,ind))
        
jogo.fps = 100
ordenados = ordenar_lista(populacao,fitness)
melhor = ordenados[0]
print('Melhor Indivíduo:', melhor)
fit = calcular_fitness(jogo,melhor)
print('Fitness: {:4.1f}'.format(jogo.get_score()))

ger | fitness
---+----------------------------------------------------------------------------
